https://www.tensorflow.org/guide/core/mlp_core

In [1]:
#! pip install tensorflow
# !pip install tensorflow_datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

2024-07-22 12:00:21.476431: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-22 12:00:21.476780: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 12:00:21.478873: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 12:00:21.485124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 12:00:21.496128: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [4]:
# Construct a tf.data.Dataset
ds = tfds.load('mnist', split='train', shuffle_files=True)

# Build your input pipeline
ds = ds.shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)
for example in ds.take(1):
  image, label = example["image"], example["label"]


2024-07-22 12:00:25.196919: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-07-22 12:00:25.197256: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
image_mlp = tf.reshape(image, [32, 28*28, 1]) # Переводим в вектор

In [8]:
class MyLayer(tf.Module):
    def __init__(self, size_input, size_output, func):
        super().__init__()
        self.size_output = size_output
        self.size_input = size_input
        self.w0 = tf.Variable(tf.random.normal([size_output, size_input],mean=0, stddev=0.001))
        # self.w1 = tf.Variable(tf.random.normal([size_output, size_input],mean=0, stddev=0.001))
        self.b = tf.Variable(tf.random.normal([size_output, 1], mean=1, stddev=0.001))
        self.sigma = func
        
    @tf.function()
    def __call__(self, tensorVecImages):
        tf.matmul(self.w )
        x2 = tf.subtract(tf.reduce_mean(hi, axis=0), 1/len(hi) * hi[0]) 
        sum_ = tf.matmul(self.w0, tf.transpose([x1]))+tf.matmul(self.w1,tf.transpose([x2]))+self.b
        y = self.sigma(sum_)
        return y

# class Model:
#     def __init__(self):
#         self.layer1 = MyLayer(6, 3, tf.nn.relu)
#         self.layer2 = MyLayer(3, 1, tf.nn.relu)
#         # self.layer1 = MyLayer(2, 2, tf.nn.relu)

#     @tf.function()
#     def __call__(self, file):
#         _, edges, _, nodes = graph_from_path(file)
#         e1 = np.array(edges[0])
#         e2 = np.array(edges[1])
#         index = [[i, *e2[e1 == i]] for i in range(len(nodes))]
#         h0 = nodes
#         h1 = [self.layer1([tf.transpose(h0[i])[0] for i in index_array]) for index_array in index]
#         h2 = [self.layer2([tf.transpose(h1[i])[0] for i in index_array]) for index_array in index]
#         return h2
            
# @tf.function()
# def loss(file, m):
#     y = m(file)
#     _, _, Y, _ = graph_from_path(file)
#     y_true = tf.transpose(np.array([[Y]], dtype=np.float32))
#     return tf.reduce_mean(tf.pow(y - y_true , 2))

In [6]:
mylayer = MyLayer(size_input=784, size_output=2, func=tf.nn.relu)

In [24]:
w0 = tf.Variable(tf.random.normal([1, 784, 1],mean=0, stddev=0.001))

In [25]:
w0.shape

TensorShape([1, 784, 1])

In [26]:
image_mlp.shape

TensorShape([32, 784, 1])

In [30]:
tf.matmul(image_mlp, w0)

NotFoundError: Could not find device for node: {{node BatchMatMulV2}} = BatchMatMulV2[T=DT_UINT8, adj_x=false, adj_y=false, grad_x=false, grad_y=false]
All kernels registered for op BatchMatMulV2:
  device='XLA_CPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, DT_COMPLEX64, DT_INT64, DT_BFLOAT16, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_GPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, DT_COMPLEX64, DT_INT64, DT_BFLOAT16, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_BFLOAT16]
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_COMPLEX128]
  device='GPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_INT64]
  device='CPU'; T in [DT_INT32]
  device='CPU'; T in [DT_INT16]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_BFLOAT16]
  device='CPU'; T in [DT_HALF]
  device='CPU'; T in [